In [2]:
path = "/data/data_user/annotations/machine_translation"
import os
os.listdir(path)

['eng_spa_pairs.tar.gz',
 'medlineplus_alignment_result.tar.gz',
 'EHR_data.tar.gz',
 'README',
 'extracted_files']

In [2]:
# import tarfile
# import os

# # Define the path to the tar.gz file
# tar_gz_path = "/data/data_user/annotations/machine_translation/EHR_data.tar.gz"

# # Extract the contents of the tar.gz file
# with tarfile.open(tar_gz_path, "r:gz") as tar:
#     tar.extractall(path="/home/mshahidul/project1/all_tran_data/data3")

# # List the extracted files
# extracted_files_path = "/home/mshahidul/project1/all_tran_data/data3"
# extracted_files = os.listdir(extracted_files_path)
# print(extracted_files)

## eng_spa_pairs.tar.gz

In [4]:
path = "/home/mshahidul/project1/all_tran_data/data1/eng_spa_pairs/medlineplus_pairs.txt"
f = open(path, "r").read().split("\n")
len(f)

144155

In [4]:
data1=[]
for x in f[100:]:
    xx=x.split("\t")
    try:
        data1.append(
            {
                "english":xx[0],
                "spain":xx[1]
            }
        )
    except:
        continue

In [5]:
path = "/home/mshahidul/project1/all_tran_data/data1/eng_spa_pairs/umass_ehr_pairs.txt"
f = open(path, "r").read().split("\n")

In [6]:
for x in f:
    xx=x.split("\t")
    try:
        data1.append(
            {
                "english":xx[0],
                "spain":xx[1]
            }
        )
    except:
        continue

## medlineplus_alignment_result.tar.gz

In [14]:
path="/home/mshahidul/project1/all_tran_data/data2/alignment_result"
data2=[]
import os
for x in os.listdir(path):
    f=open(path+"/"+x,"r").read().split("\n")
    ## data format
    # l1: Poisoning is caused by swallowing, injecting, breathing in, or otherwise being exposed to a harmful substance.
    # l2: Una intoxicación (envenenamiento) es causada por la ingestión, inyección, inhalación o cualquier exposición a una sustancia dañina.

    for xx in range(0,len(f)-1,2):
        data2.append(
            {
                "english":f[xx].split("l1: ")[1],
                "spain":f[xx+1].split("l2: ")[1]
            }
        )


In [15]:
len(data2)

144154

## EHR_data.tar.gz

In [1]:
data3=[]

In [3]:
path="/home/mshahidul/project1/all_tran_data/data3/EHR/google_result"
import os
for x in os.listdir(path):
    f=open(path+"/"+x,"r").read().split("\n")
    

    for xx in range(0,len(f)-1,3):
        data3.append(
            {
                "english":f[xx].split("l1: ")[1],
                "spain":f[xx+1].split("l2: ")[1]
            }
        )

In [5]:
len(data3)

54

In [6]:
path="/home/mshahidul/project1/all_tran_data/data3/EHR/moses_result"
for x in os.listdir(path):
    f=open(path+"/"+x,"r").read().split("\n")

    for xx in range(0,len(f)-1,3):
        data3.append(
            {
                "english":f[xx].split("l1: ")[1],
                "spain":f[xx+1].split("l2: ")[1]
            }
        )

In [11]:
import pandas as pd

# Convert data3 to a DataFrame
df_data3 = pd.DataFrame(data3)

# Save the DataFrame to an Excel file
df_data3.to_excel('/home/mshahidul/project1/all_tran_data/data3.xlsx', index=False)

print("data3 has been saved to data3.xlsx")

data3 has been saved to data3.xlsx


## Accumulate all the data

In [12]:
data=data1+data2+data3

In [17]:
sent=[x['english'] for x in data]

In [13]:
# import json
# with open("/home/mshahidul/project1/all_tran_data/eng_spa_all_pairs.json", "w") as outfile:
#     json.dump(data, outfile)

In [14]:
## json load
# import json
# with open("/home/mshahidul/project1/all_tran_data/eng_spa_all_pairs.json") as f:
#     data = json.load(f)

In [17]:
import random
from collections import defaultdict

output_file = 'MedlinePlus.txt'
total_samples = 1000
sentences = [x['english'] for x in data2]

# Here, we use a simple binning: every 5 words constitutes a bin.
bins = defaultdict(list)
for sentence in sentences:
    word_count = len(sentence.split())
    bin_key = word_count // 5  # adjust the divisor to change bin width if desired
    bins[bin_key].append(sentence)

# Determine the number of bins and how many samples per bin (using equal allocation)
num_bins = len(bins)
samples_per_bin = total_samples // num_bins

sampled_sentences = []

# For each bin, sample up to samples_per_bin sentences.
for bin_key, sents in bins.items():
    if len(sents) <= samples_per_bin:
        # If the bin has fewer sentences than desired, take them all.
        sampled_sentences.extend(sents)
    else:
        sampled_sentences.extend(random.sample(sents, samples_per_bin))

# If we haven't reached 1,000 sentences yet, sample additional sentences randomly
if len(sampled_sentences) < total_samples:
    remaining_needed = total_samples - len(sampled_sentences)
    # Get the remaining sentences that were not sampled
    already_sampled = set(sampled_sentences)
    remaining = [s for s in sentences if s not in already_sampled]
    if len(remaining) < remaining_needed:
        sampled_sentences.extend(remaining)
    else:
        sampled_sentences.extend(random.sample(remaining, remaining_needed))
elif len(sampled_sentences) > total_samples:
    sampled_sentences = random.sample(sampled_sentences, total_samples)

# Write the sampled sentences to the output file
with open(output_file, 'w', encoding='utf-8') as f:
    for sentence in sampled_sentences:
        f.write(sentence + "\n")

print(f"Successfully saved {len(sampled_sentences)} sentences with varied lengths to {output_file}")


Successfully saved 1000 sentences with varied lengths to MedlinePlus.txt


In [ ]:
# Create a dictionary from data2
english_to_spanish = {item['english']: item['spain'] for item in data2}

# Find the corresponding Spanish sentences for sampled_sentences
sampled_spanish_sentences = [{"english":sentence,"spanish":english_to_spanish[sentence]} for sentence in sampled_sentences if sentence in english_to_spanish]

# Print the corresponding Spanish sentences
for sentence in sampled_spanish_sentences:
    print(sentence)

In [23]:
import json

# Define the output file path
output_file = "/home/mshahidul/project1/code/Sampled_1000_MedlinePlus_eng_spanish_pair.json"

# Save to JSON file
with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(sampled_spanish_sentences, json_file, ensure_ascii=False, indent=4)

print(f"Sampled translations saved successfully to {output_file}")


Sampled translations saved successfully to /home/mshahidul/project1/code/Sampled_1000_MedlinePlus_eng_spanish_pair.json
